In [ ]:

from ultralytics import YOLO
import os
import numpy as np
from PIL import Image
import pandas as pd
from supervision.detection.utils import box_iou_batch
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# This script containes all the code for IOU and AP computation
from utils import compute_iou, Compute_AP, generate_boxes,Compute_AP_withboxes

# Q1

Write a function to compute IoU (Intersection over Union) between two axis-aligned bounding boxes specified in the Ultralytics YOLO format.

You MUST use the shapely library and its functionalities to write your function. 

Show that your function provides the same or similar answer as IoU computed using `supervision` library 


In [ ]:
# Load YOLO model
model = YOLO("runs/detect/train10/weights/best.pt")
test_img_dir = "data/test/images"
test_label_dir = "data/test/labels"

# Load test images and labels
test_images = np.sort(os.listdir(test_img_dir))
test_labels = np.sort(os.listdir(test_label_dir))

imgs = [Image.open(os.path.join(test_img_dir, img)) for img in test_images]

gt_box = []
for label in test_labels:
    data = pd.read_csv(os.path.join(test_label_dir, label), header=None, sep=" ")
    boxes = []
    
    # Converting xc, yc, w, h to x1, y1, x2, y2
    for _, row in data.iterrows():
        xc, yc, w, h = row[1:].values 
        x1 = (xc - w / 2) * 416
        y1 = (yc - h / 2) * 416
        x2 = (xc + w / 2) * 416
        y2 = (yc + h / 2) * 416
        boxes.append([x1, y1, x2, y2]) 
    
    gt_box.append(np.array(boxes))  

# Run YOLO model
results = model(imgs, verbose=False)

# delete the imgs to save space
del imgs

pred_box = [result.boxes.xyxy.cpu().numpy() for result in results]

In [62]:
# Compute IoU using `shapely` library
iou_values = []
for gt, pred in zip(gt_box, pred_box):  

    if len(pred) == 0 or len(gt) == 0:
        continue

    # Extract the IoU scores
    iou_matrix,_ = compute_iou(pred, gt)
    max_ious = np.max(iou_matrix, axis=0) 
    iou_values.extend(max_ious)

print(np.nanmean(iou_values))  # Print mean IoU

0.7984642933999669


In [63]:
# compute using the supervision library
iou_scores = []
for gt, pred in zip(gt_box, pred_box):  

    if len(pred) == 0 or len(gt) == 0:
        continue

    # Extract the IoU scores
    iou_matrix = box_iou_batch(gt, pred)  
    max_ious = np.max(iou_matrix, axis=1) 
    iou_scores.extend(max_ious)

print(np.nanmean(iou_scores))

0.798464293117701


# Q2
### Write a function to compute Average Precision (AP)

a) Use Pascal VOC 11 point interpolation method to implement the function 

b) Use COCO 101-point interpolation method to implement the function 

c) Use Area under Precision-Recall Curve (AP) method to implement the function 

In [14]:
ap , _ = Compute_AP(model, test_images, test_labels, r_levels="Pascal_VOC11" , method="Interpolation")
print("Precision calculated using Pascal VOC 11 Interpolation criterion: ", ap)

ap , _ = Compute_AP(model, test_images, test_labels, r_levels="Pascal_VOC11" , method="Area_Under_Curve")
print("Precision calculated using Pascal VOC 11 Area Under Curve criterion: ", ap)

ap , _ = Compute_AP(model, test_images, test_labels, r_levels="COCO_101" , method="Interpolation")
print("Precision calculated using COCO 101 Interpolation criterion: ", ap)

ap , _ = Compute_AP(model, test_images, test_labels, r_levels="COCO_101" , method="Area_Under_Curve")
print("Precision calculated using COCO 101 Area Under Curve criterion: ", ap)

Precision calculated using Pascal VOC 11 Interpolation criterion:  0.9795082563390765
Precision calculated using Pascal VOC 11 Area Under Curve criterion:  0.9868111473955046
Precision calculated using COCO 101 Interpolation criterion:  0.9819969832421995
Precision calculated using COCO 101 Area Under Curve criterion:  0.9825950238853318


d) Randomly generate 10 images of size 100x100. Randomly generate 10 ground truth boxes of size 20x20 and 10 predicted boxes of size 20x20 in each image. 

Assume there is only one class of objects. Compare the AP50 (Average Precision at IoU 0.5) computed by 3 of your methods

In [28]:
img_sz = (100,100)

random_gt = generate_boxes(10, img_sz,box_size=20)
random_pred = generate_boxes(10, img_sz,box_size=20)
conf = np.random.rand(len(random_pred))

In [29]:
ap, _ = Compute_AP_withboxes(random_pred, random_gt, conf, r_levels="Pascal_VOC11" , method="Interpolation")
print("Precision calculated using Pascal VOC 11 Interpolation criterion: ", ap)

ap, _ = Compute_AP_withboxes(random_pred, random_gt, conf, r_levels="Pascal_VOC11" , method="Area_Under_Curve")
print("Precision calculated using Pascal VOC 11 Area Under Curve criterion: ", ap)

ap, _ = Compute_AP_withboxes(random_pred, random_gt, conf, r_levels="COCO_101" , method="Interpolation")
print("Precision calculated using COCO 101 Interpolation criterion: ", ap)

ap, _ = Compute_AP_withboxes(random_pred, random_gt, conf, r_levels="COCO_101" , method="Area_Under_Curve")
print("Precision calculated using COCO 101 Area Under Curve criterion: ", ap)

Precision calculated using Pascal VOC 11 Interpolation criterion:  0.11688311688311687
Precision calculated using Pascal VOC 11 Area Under Curve criterion:  0.0761904761904762
Precision calculated using COCO 101 Interpolation criterion:  0.13154172560113153
Precision calculated using COCO 101 Area Under Curve criterion:  0.011619047619047612


### Random Trials

In [36]:
n_trials = 10000

results_df = pd.DataFrame(columns=["Pascal_VOC11_Interpolation", "Pascal_VOC11_Area_Under_Curve", "COCO_101_Interpolation", "COCO_101_Area_Under_Curve"])

for i in range(n_trials):
    random_gt = generate_boxes(10, img_sz,box_size=20)
    random_pred = generate_boxes(10, img_sz,box_size=20)
    conf = np.random.rand(len(random_pred))

    ap, _ = Compute_AP_withboxes(random_pred, random_gt, conf, r_levels="Pascal_VOC11" , method="Interpolation")
    results_df.loc[i, "Pascal_VOC11_Interpolation"] = ap

    ap, _ = Compute_AP_withboxes(random_pred, random_gt, conf, r_levels="Pascal_VOC11" , method="Area_Under_Curve")
    results_df.loc[i, "Pascal_VOC11_Area_Under_Curve"] = ap

    ap, _ = Compute_AP_withboxes(random_pred, random_gt, conf, r_levels="COCO_101" , method="Interpolation")
    results_df.loc[i, "COCO_101_Interpolation"] = ap

    ap, _ = Compute_AP_withboxes(random_pred, random_gt, conf, r_levels="COCO_101" , method="Area_Under_Curve")
    results_df.loc[i, "COCO_101_Area_Under_Curve"] = ap

In [38]:
results_df.mean(axis=0)  # Print mean AP for each criterion

Pascal_VOC11_Interpolation       0.077653
Pascal_VOC11_Area_Under_Curve    0.048609
COCO_101_Interpolation           0.059844
COCO_101_Area_Under_Curve        0.005438
dtype: object